<a href="https://colab.research.google.com/github/jjangmo91/ParkLab/blob/main/project/carbon_sink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. 기본 설정 (Setup)

In [ ]:
import ee
import geemap

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-jjangmo91')

# 분석 지역 정의 (속리산 국립공원, WDPA ID: 773)
wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")
roi = wdpa.filter(ee.Filter.eq('WDPAID', 773)).geometry()

# 데이터 로드
dem = ee.Image("NASA/NASADEM_HGT/001").select('elevation').clip(roi)
slope = ee.Terrain.slope(dem)
lulc = ee.ImageCollection("ESA/WorldCover/v200").first().clip(roi)
boundary = ee.Image().paint(roi, 0, 1)
boundary_vis = {'palette': ['#000000']} # 검은색

# 1단계: 현재 토지 피복 상태 확인 (문제 원인 파악용)

In [ ]:
map0 = geemap.Map()
map0.centerObject(roi, 11)
map0.add_basemap('HYBRID')

def create_sld_style(class_values, palette, labels):
    sld = '<RasterSymbolizer><ColorMap type="values" extended="false">'
    for value, color, label in zip(class_values, palette, labels):
        sld += f'<ColorMapEntry color="{color}" quantity="{value}" label="{label}" />'
    sld += '</ColorMap></RasterSymbolizer>'
    return sld

class_values = [10, 20, 30, 40, 50, 60, 80, 90]
class_palette = ["#006400", "#ffbb22", "#ffff4c", "#f096ff", "#fa0000", "#b4b4b4", "#0064c8", "#0096a0"]
class_labels = ["산림", "관목지", "초지", "농경지", "시가화", "나지", "수역", "습지"]

sld_style = create_sld_style(class_values, class_palette, class_labels)
map0.addLayer(lulc.sldStyle(sld_style), {}, '현재 토지 피복 (ESA WorldCover)')
map0.addLayer(boundary, boundary_vis, '국립공원 경계')

legend_dict_lulc = {label: color for label, color in zip(class_labels, class_palette)}
map0.add_legend(title="현재 토지 피복", legend_dict=legend_dict_lulc)
print("지도 0: 속리산의 전체 토지 피복 유형과 범례가 정확히 일치하는 지도입니다.")
display(map0)

# 2. 제외 지역 분석 (Exclusion Analysis)

In [ ]:
existing_forest = lulc.eq(10)
water_bodies = lulc.eq(80)
alpine_zone = dem.gte(950)
bare_land = lulc.eq(60)
steep_slopes_on_bare = bare_land.And(slope.gte(35))
exclusion_mask = existing_forest.Or(water_bodies).Or(alpine_zone).Or(steep_slopes_on_bare)
potential_candidate_area = exclusion_mask.Not()

map1 = geemap.Map()
map1.centerObject(roi, 12)
map1.add_basemap('HYBRID')
map1.addLayer(exclusion_mask.selfMask(), {'palette': ['#696969']}, '제외 지역 (조림 불가)')
map1.addLayer(potential_candidate_area.selfMask(), {'palette': ['#ffffbf']}, '1차 후보지 (분석 대상)')
map1.addLayer(boundary, boundary_vis, '국립공원 경계')
map1.add_legend(title="1단계: 제외 분석 결과", legend_dict={ "제외 지역": "#696969", "1차 후보지": "#ffffbf" })
display(map1)

# 3. 적합성 평가 (Suitability Assessment)

In [ ]:
# 적합성 평가에 필요한 모든 변수 계산

# 개별 점수 계산
slope_score = slope.unitScale(0, 35).subtract(1).multiply(-1)
soc_dataset = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').clip(roi)
soil_score = soc_dataset.unitScale(0, 100)
npp_dataset = ee.ImageCollection('MODIS/061/MOD17A3HGF').filterDate('2015-01-01', '2022-12-31').select('Npp').mean().clip(roi)
productivity_score = npp_dataset.unitScale(0, 15000)

# 종합 점수 계산(검증용 & 최종 분석용)
suitability_score_full_area = (slope_score.add(soil_score).add(productivity_score)).divide(3)
suitability_score_masked = suitability_score_full_area.updateMask(potential_candidate_area)

# 시각화에 사용할 파라미터 정의
suitability_palette = ['#d7191c', '#fdae61', '#ffffbf', '#abdda4', '#2b83ba']
suitability_vis_params = {'min': 0, 'max': 1, 'palette': suitability_palette}

In [ ]:
# 지도 2-A: 검증용 지도 (국립공원 전체)
map2_verification = geemap.Map()
map2_verification.centerObject(roi, 12)
map2_verification.add_basemap('HYBRID')

# 종합 적합성(전체 지역) 레이어 추가
map2_verification.addLayer(
    suitability_score_full_area,
    suitability_vis_params,
    'Suitability Score (Full Area)'
)
# 국립공원 경계선 추가
map2_verification.addLayer(boundary, boundary_vis, 'National Park Boundary')
map2_verification.add_colorbar(
    suitability_vis_params,
    label="Suitability Score (Low -> High)",
    orientation="vertical",
    max_width="100px"
)

print("Map 2-A (Verification): Suitability score for the entire national park.")
display(map2_verification)


In [ ]:
# 실제 분석 결과 지도
map2_final = geemap.Map()
map2_final.centerObject(roi, 12)
map2_final.add_basemap('HYBRID')

# 종합 적합성(1차 후보지) 레이어 추가
map2_final.addLayer(
    suitability_score_masked,
    suitability_vis_params,
    'Suitability Score (Candidate Area)'
)
# 국립공원 경계선 추가
map2_final.addLayer(boundary, boundary_vis, 'National Park Boundary')
map2_final.add_colorbar(
    suitability_vis_params,
    label="Suitability Score (Low -> High)",
    orientation="vertical",
    max_width="100px"
)

print("Map 2-B (Final Analysis): Suitability score for the candidate areas only.")
display(map2_final)

# 4. 우선순위 부여 (Prioritization)

In [ ]:
# 우선순위 평가에 필요한 모든 변수 계산

# 생태계 연결성 점수 계산
distance_to_forest = existing_forest.distance(ee.Kernel.euclidean(1000, 'meters'))
connectivity_score = distance_to_forest.unitScale(0, 1000).subtract(1).multiply(-1)

# 복원 시급성 점수 계산
def get_annual_ndvi(year):
    def harmonize_bands(image):
        optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
        landsat_8_9_list = ee.List(['LANDSAT_8', 'LANDSAT_9'])
        is_landsat_8_9 = landsat_8_9_list.contains(image.get('SPACECRAFT_ID'))
        common_bands = ee.Image(ee.Algorithms.If(
            is_landsat_8_9,
            optical_bands.select(['SR_B4', 'SR_B5'], ['Red', 'NIR']),
            optical_bands.select(['SR_B3', 'SR_B4'], ['Red', 'NIR'])
        ))
        return image.addBands(common_bands, overwrite=True)

    collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').merge(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')) \
                   .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')).merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2'))

    filtered = collection.filterDate(ee.Date.fromYMD(year, 6, 1), ee.Date.fromYMD(year, 8, 31)).filterBounds(roi).map(harmonize_bands)

    return ee.Image(ee.Algorithms.If(
        filtered.size().gt(0),
        filtered.median().normalizedDifference(['NIR', 'Red']).rename('NDVI').set('year', year),
        None
    ))

years = ee.List.sequence(2020, 2024)
ndvi_collection = ee.ImageCollection.fromImages(years.map(lambda y: get_annual_ndvi(ee.Number(y))).removeAll([None]))
def add_time_band(image):
    return image.addBands(ee.Image.constant(image.get('year')).toFloat().rename('time'))
trend = ndvi_collection.map(add_time_band).select(['time', 'NDVI']).reduce(ee.Reducer.linearFit())
ndvi_slope = trend.select('scale')
restoration_urgency_score = ndvi_slope.unitScale(-0.01, 0.005).subtract(1).multiply(-1)

# 시각화에 사용할 파라미터 정의
priority_palette = ['#ffffcc', '#a1dab4', '#41b6c4'] # 낮음(노랑) -> 높음(청록)
priority_vis_params = {'min': 0, 'max': 1, 'palette': priority_palette}

print("-> 모든 우선순위 점수 계산 완료. 아래 셀에서 지도를 확인하세요.")

In [ ]:
# 우선순위 요소 시각화
map3 = geemap.Map()
map3.centerObject(roi, 12)
map3.add_basemap('HYBRID')

# '생태계 연결성' 레이어 추가
map3.addLayer(
    connectivity_score.updateMask(potential_candidate_area),
    priority_vis_params,
    'Ecological Connectivity Score'
)
# '복원 시급성' 레이어 추가
map3.addLayer(
    restoration_urgency_score.updateMask(potential_candidate_area),
    priority_vis_params,
    'Restoration Urgency Score',
    False # 레이어를 처음에는 끈 상태로 추가
)
# 국립공원 경계선 추가
map3.addLayer(boundary, boundary_vis, 'National Park Boundary')

# 범례
map3.add_colorbar(
    priority_vis_params,
    label="Priority Score (Low -> High)",
    orientation="vertical",
    max_width="100px"
)

print("Map 3: Displays priority scores. You can toggle layers in the layer manager.")
display(map3)


# 5. 최종 종합 및 시각화

In [ ]:
# 최종 결과 종합

# 최종 점수 계산 (가중 중첩)
# suitability_score_masked는 2단계에서 계산된 변수를 그대로 사용
final_score = (
    suitability_score_masked.multiply(0.5)
    .add(connectivity_score.multiply(0.3))
    .add(restoration_urgency_score.multiply(0.2))
)

# 최종 후보지 등급 분류 (1, 2, 3 등급)
priority_zones = (
    ee.Image(0)
    .where(final_score.gte(0.7), 1)  # 1순위 (최우선)
    .where(final_score.gte(0.5).And(final_score.lt(0.7)), 2) # 2순위 (차순위)
    .where(final_score.gt(0).And(final_score.lt(0.5)), 3) # 3순위 (후순위)
    .updateMask(potential_candidate_area)
)
print("-> 최종 점수 및 등급 계산 완료. 아래 셀에서 지도를 확인하세요.")

#최종 후보지 시각화
map4 = geemap.Map()
map4.centerObject(roi, 12)
map4.add_basemap('HYBRID')

# 최종 우선순위 등급 지도 시각화
final_palette = ['#d73027', '#fc8d59', '#fee090'] # 1순위(진홍), 2순위(주황), 3순위(노랑)
map4.addLayer(
    priority_zones,
    {'min': 1, 'max': 3, 'palette': final_palette},
    'Final Priority Zones'
)

# 제외 지역 및 국립공원 경계선 추가
map4.addLayer(exclusion_mask.selfMask(), {'palette': ['#696969']}, 'Excluded Areas', False)
map4.addLayer(boundary, boundary_vis, 'National Park Boundary')

# 범례
map4.add_legend(title="Final Priority Zones", legend_dict={
    "Priority 1 (Highest)": "#d73027",
    "Priority 2 (Medium)": "#fc8d59",
    "Priority 3 (Lowest)": "#fee090",
    "Excluded Areas": "#696969"
})

display(map4)